# Calculatar 重新版

In [1]:
from IPython.display import display, HTML, clear_output
from ipywidgets import Button, IntText, VBox, Output, Dropdown
from random import randint
from PIL import Image, ImageDraw
import expressions
import random


# Language selection
languages = {
    'English': {
        'question': ' = ?',
        'correct': 'Correct',
        'almost': 'Almost',
        'cheer_up': 'Cheer up',
        'hint_button': 'Hint',
        'circles': 'How many {0} do you have?',
        'answer_button': 'Answer',  
    },
    'Japanese': {
        'question': ' = ?',
        'correct': '正解',
        'almost': 'ほぼ正解',
        'cheer_up': '頑張りましょう',
        'hint_button': 'ヒント',
        'circles': '{0}は何個ありますか？',
        'answer_button': '答え',  
    },
    'Chinese': {
        'question': ' = ?',
        'correct': '正确',
        'almost': '几乎正确',
        'cheer_up': '加油',
        'hint_button': '提示',
        'circles': '有多少个 {0}？',
        'answer_button': '答案',
    }
}

In [2]:
# Language selection dropdown
language_dropdown = Dropdown(
    options=languages.keys(),
    description='Select Language:',
    layout={'width': '200px'}
)

display(language_dropdown)

Dropdown(description='Select Language:', layout=Layout(width='200px'), options=('English', 'Japanese', 'Chines…

# Select Language 言語選択

In [3]:
# Generate arithmetic expressions based on selected language
selected_language = language_dropdown.value

# Function to print selected language
def print_selected_language(change):
    global selected_language
    selected_language = change['new']
    print(f"Selected Language: {selected_language}")

# Call the function to print selected language when the dropdown value changes
language_dropdown.observe(print_selected_language, names='value')

# Call the function to print the initial selected language
print_selected_language({'new': selected_language})

Selected Language: Japanese


# Generate step

In [4]:
# Create widgets and layout
operators = ['+', '-', '*', '/']
expressions = []

while len(expressions) < 5:
    num1 = random.randint(1, 10)
    num2 = random.randint(1, 10)
    operator = random.choice(operators)
    if operator == '/':
        # Ensure integer division result
        num1 = num1 * num2
    expression = f"{num1} {operator} {num2}"
    result = eval(expression)
    # Ensure the result is an integer and less than 100
    if isinstance(result, int) and result < 100:
        expressions.append((expression, result))

# Create layout step

In [5]:
# Hint button click event
def hint_button_clicked(b):
    index = hint_buttons.index(b)
    with output:
        num_circles = hint_textboxes[index].value
        expression = expressions[index]
        operand1, operator, operand2 = expression[0].split()
        operator = operator_conversion(operator)  # Convert operator symbol if needed
        circle_marks = get_circle_marks(num_circles, operand1, operator, operand2)
        expression_hint = f"{operand1} {operator} {operand2}: {circle_marks}"
        hint_textboxes[index].description = expression_hint

# Hint (if you know, pass this step

In [6]:
# Operator conversion function
def operator_conversion(operator):
    if operator == '+':
        return '＋'
    elif operator == '-':
        return '－'
    elif operator == '*':
        return '×'
    elif operator == '/':
        return '÷'
    else:
        return operator

# Get circle marks based on calculation
def get_circle_marks(num_circles, operand1, operator, operand2):
    result = calculate_result(operand1, operator, operand2)
    if result is not None:
        result_str = str(result)
        circle_marks = "〇" * len(result_str) + "●" * (num_circles - len(result_str))
        return circle_marks
    else:
        return "Invalid calculation"

# Add click events to hint buttons

In [7]:
# Calculate result function
def calculate_result(operand1, operator, operand2):
    try:
        result = eval(f"{operand1} {operator} {operand2}")
        return result
    except Exception as e:
        print(f"Error in calculation: {e}")
        return None


# Generate circle image(普通算法）

In [8]:
# Define the hint generation function
def generate_hint(operand1, operator, operand2):
    # Convert operands to integers
    operand1 = int(operand1)
    operand2 = int(operand2)
    # Generate hint based on operator
    if operator == '＋':
        return '●' * operand1 + ' ' + operator + ' ' + '〇' * operand2
    elif operator == '－':
        return '●' * operand1 + ' ' + operator + ' ' + '〇' * operand2
    elif operator == '×':
        return ' + '.join(['●' * operand1 for _ in range(operand2)])
    elif operator == '÷':
        if operand2 != 0:
            return ' - '.join(['●' * operand2 for _ in range(operand1 // operand2)])

# Define the button click event
def hint_button_clicked(b):
    index = hint_buttons.index(b)
    with output:
        num_circles = hint_textboxes[index].value
        expression = expressions[index]
        operand1, operator, operand2 = expression[0].split()
        operator = operator_conversion(operator)  # Convert operator symbol if needed
        hint = generate_hint(operand1, operator, operand2)
        print(hint)  # Print the hint in the output box



# Answer button click event
def answer_button_clicked(b):
    correct_answers = 0
    total_questions = len(expressions)

    with output:
        for i in range(total_questions):
            expression = expressions[i]
            result = expression[1] # The result of the expression

            if hint_textboxes[i].value == result: # If the input value matches the result
                correct_answers += 1
                print(f"Answer for {expression[0]}: {result} is Correct")

            else:
                print(f"Answer for {expression[0]}: {result} is Incorrect")

        score = (correct_answers / total_questions) * 100

        if score == 100:
            print(languages[selected_language]['correct'])
        elif score >= 70:
            print(languages[selected_language]['almost'])
        else:
            print(languages[selected_language]['cheer_up'])


        
# Create layout
output = Output()
hint_buttons = []
hint_textboxes = []
answer_buttons = []  # Initialize answer_buttons list        
        
        
for i in range(5):
    hint_button = Button(description=languages[selected_language]['hint_button'])
    hint_button.on_click(hint_button_clicked)
    hint_buttons.append(hint_button)
    


    hint_textbox = IntText(value=0, description=expressions[i][0] + languages[selected_language]['question'])
    hint_textboxes.append(hint_textbox)


# Create an answer button
answer_button = Button(description=languages[selected_language]['answer_button'])
answer_button.on_click(answer_button_clicked)

# Flatten the list of tuples into a single list of widgets (without answer_buttons)
widgets = [widget for pair in zip(hint_textboxes, hint_buttons) for widget in pair]

# Add the answer_button at the end of the widgets list
widgets.append(answer_button)

layout = VBox(widgets, layout={'width': '400px'})

# Display layout
display(layout)
display(output)


Output()

# Addtional calculation

In [9]:
from IPython.display import display, HTML, clear_output
from ipywidgets import Button, IntText, VBox, Output, Dropdown
from random import randint
from PIL import Image, ImageDraw
import expressions
import random

def generate_expressions(num_expressions):
    expressions = []

    for _ in range(num_expressions):
        operator = random.choice(['+', '-', '*', '/'])

        if operator in ['+', '-']:
            total = random.randint(2, 10)  # Ensure total is at least 2
            operand = random.randint(1, total - 1)  # Pick operand from 1 to total-1 to avoid zero and self subtraction
            missing_operand = total - operand if operator == '+' else total - operand
            expression = f"{total} = ? {operator} {operand}"
        else:
            operand1 = random.randint(1, 10)
            operand2 = random.randint(1, 10)

            if operator == '*':
                total = operand1 * operand2
                expression = f"{total} = {operand1} {operator} ?"
                missing_operand = operand2
            else:  # operator == '/'
                total = operand1 * operand2
                expression = f"{total} = ? {operator} {operand1}"
                missing_operand = operand2  # 'missing_operand' should be operand2

        expressions.append((expression, missing_operand))

    return expressions


In [10]:
def hint_button_clicked(b):
    index = hint_buttons.index(b)
    with output:
        clear_output()
        expression, missing_operand = expressions[index]
        parts = expression.split()
        operator = parts[3]

        # Determine if '?' is in the position of the first operand or the second operand
        if parts[2] == '?':
            known_operand = int(parts[4])  # If '?' is at the 2nd position, the known operand is at the 4th position
            additional = False
        else:
            known_operand = int(parts[2])  # If '?' is at the 4th position, the known operand is at the 2nd position
            additional = True

        hint = generate_hint(known_operand, operator, known_operand, additional)
        print(hint)

# Answer button click event
def answer_button_clicked(b):
    correct_answers = 0
    total_questions = len(expressions)

    with output:
        for i in range(total_questions):
            expression, result = expressions[i]  # The answer is already generated and saved in 'result'
            if int(hint_textboxes[i].value) == result:  # Convert the input value to integer before comparison
                correct_answers += 1
                print(f"Answer for {expression}: {result} is Correct")
            else:
                print(f"Answer for {expression}: {result} is Incorrect")

        score = (correct_answers / total_questions) * 100
        if score == 100:
            print(languages[selected_language]['correct'])
        elif score >= 70:
            print(languages[selected_language]['almost'])
        else:
            print(languages[selected_language]['cheer_up'])


#  Generate circle image(逆算法）

In [11]:
      
# Define the hint generation function
def generate_hint(known_operand, operator, missing_operand, additional):
    known_operand = int(known_operand)
    if operator == '+':
        if additional:
            return '●' * known_operand + ' ' + operator + ' ' + '〇' * missing_operand
        else:
            return '〇' * missing_operand + ' ' + operator + ' ' + '●' * known_operand
    elif operator == '-':
        if additional:
            return '〇' * missing_operand + ' ' + operator + ' ' + '●' * known_operand
        else:
            return '●' * known_operand + ' ' + operator + ' ' + '〇' * missing_operand
    elif operator == '*':
        return ' + '.join(['●' * known_operand for _ in range(missing_operand)])
    elif operator == '/':
        if missing_operand != 0:
            return ' - '.join(['●' * missing_operand for _ in range(known_operand // missing_operand)])
        else:
            return 'Can not divide by zero.'
    return ''


# Global variable initializations
expressions = generate_expressions(5)
hint_buttons = []
hint_textboxes = []

# Widget creation and event handlers
for i in range(5):
    hint_button = Button(description=languages[selected_language]['hint_button'])
    hint_button.on_click(hint_button_clicked)
    hint_buttons.append(hint_button)
    
    hint_textbox = IntText(value=0, description=expressions[i][0] + languages[selected_language]['question'])
    hint_textboxes.append(hint_textbox)

answer_button = Button(description=languages[selected_language]['answer_button'])
answer_button.on_click(answer_button_clicked)

# Displaying widgets
widgets = [widget for pair in zip(hint_textboxes, hint_buttons) for widget in pair]
widgets.append(answer_button)

layout = VBox(widgets, layout={'width': '400px'})

display(layout)
display(output)

Output(outputs=({'name': 'stdout', 'text': '●●●●●●●● + ●●●●●●●● + ●●●●●●●● + ●●●●●●●● + ●●●●●●●● + ●●●●●●●● + …